# Data Preprocessing

In [1]:
# import pandas as pd 
# from scipy import stats
# from sklearn.impute import SimpleImputer
# from sklearn.preprocessing import LabelEncoder
# from sklearn.preprocessing import StandardScaler
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.decomposition import PCA
# from sklearn.feature_selection import SelectKBest, f_classif

In [2]:
# Core libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import zipfile
import warnings
warnings.filterwarnings('ignore')

# Preprocessing libraries
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif
from sklearn.ensemble import IsolationForest
from sklearn.metrics import classification_report, confusion_matrix

# Statistical libraries
from scipy import stats
from scipy.stats import zscore, skew

# Set style for better visualizations
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Libraries imported successfully!")


Libraries imported successfully!


In [3]:
from category_encoders import BinaryEncoder
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, OrdinalEncoder

## Data Cleaning

In [4]:
train_url = r'https://raw.githubusercontent.com/ek-chris/Practice_datasets/refs/heads/main/home_loan_train.csv'
test_url = r'https://raw.githubusercontent.com/ek-chris/Practice_datasets/refs/heads/main/home_loan_test.csv'

In [5]:
test_data = pd.read_csv(test_url, sep=",")
test_data.head(2)

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,LP001015,Male,Yes,0,Graduate,No,5720,0,110.0,360.0,1.0,Urban
1,LP001022,Male,Yes,1,Graduate,No,3076,1500,126.0,360.0,1.0,Urban


In [6]:
train_data = pd.read_csv(train_url, sep=",")
train_data.head(2)

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N


In [7]:
df = train_data.copy()

In [8]:
#Correcting datatypes
# df["Dependents"] = df["Dependents"].replace("3+", 3)

df= df.astype({"ApplicantIncome":"float64"})

In [9]:
#Checking missing values
print("Missing Values")
missing_values = df.isnull().sum()
if missing_values.sum()>0:
    print(missing_values[missing_values > 0])
else:
    print("No missing values")

#Checking for duplicates
print("\nDuplicate Rows:")
duplicates = df.duplicated().sum()
print(f"Number of duplicated Rows: {duplicates}")

if duplicates > 0:
    print(f"Percentage of duplicates: {(duplicates/len(duplicates))*100:.2f}%")
else:
    print("There are no duplicate rows.")

Missing Values
Gender              13
Married              3
Dependents          15
Self_Employed       32
LoanAmount          22
Loan_Amount_Term    14
Credit_History      50
dtype: int64

Duplicate Rows:
Number of duplicated Rows: 0
There are no duplicate rows.


In [10]:
#Handling missing values
df.dropna(subset=["LoanAmount"], inplace=True)

In [11]:
df.dropna(subset=["Loan_Amount_Term"], inplace=True)

In [12]:
#Handling missing values
df["Gender"] = df['Gender'].fillna(df["Gender"].mode().iloc[0])
df["Married"] = df['Married'].fillna(df["Married"].mode().iloc[0])
df["Self_Employed"] = df['Self_Employed'].fillna(df["Self_Employed"].mode().iloc[0])
df["Dependents"] = df['Dependents'].fillna(df["Dependents"].mode().iloc[0])

df["Credit_History"] = df['Credit_History'].fillna(0)


In [13]:
be = BinaryEncoder()
le = LabelEncoder()

In [14]:
df["Loan_Status"] = le.fit_transform(df["Loan_Status"])
df[["Loan_Status"]]

,Loan_Status
1,0
2,1
3,1
4,1
5,1
...,...
609,1
610,1
611,1
612,1


In [15]:
# df["Loan_Status_Int"] = (df["Loan_Status"] == 'Y').astype(int)

In [16]:
# def map_credit_history_category(c):
#     """
#     Converts credit history to a categorical feature
#     """
#     if c == 1:
#         return "Good"
#     else:
#         return "Bad"

In [17]:
# df["Credit_History_Label"] = df["Credit_History"].apply(map_credit_history_category)

In [18]:
df.isnull().sum()

Loan_ID              0
Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 578 entries, 1 to 613
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            578 non-null    object 
 1   Gender             578 non-null    object 
 2   Married            578 non-null    object 
 3   Dependents         578 non-null    object 
 4   Education          578 non-null    object 
 5   Self_Employed      578 non-null    object 
 6   ApplicantIncome    578 non-null    float64
 7   CoapplicantIncome  578 non-null    float64
 8   LoanAmount         578 non-null    float64
 9   Loan_Amount_Term   578 non-null    float64
 10  Credit_History     578 non-null    float64
 11  Property_Area      578 non-null    object 
 12  Loan_Status        578 non-null    int64  
dtypes: float64(5), int64(1), object(7)
memory usage: 63.2+ KB


## 2. EDA-Based Data Quality Assessment

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 578 entries, 1 to 613
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            578 non-null    object 
 1   Gender             578 non-null    object 
 2   Married            578 non-null    object 
 3   Dependents         578 non-null    object 
 4   Education          578 non-null    object 
 5   Self_Employed      578 non-null    object 
 6   ApplicantIncome    578 non-null    float64
 7   CoapplicantIncome  578 non-null    float64
 8   LoanAmount         578 non-null    float64
 9   Loan_Amount_Term   578 non-null    float64
 10  Credit_History     578 non-null    float64
 11  Property_Area      578 non-null    object 
 12  Loan_Status        578 non-null    int64  
dtypes: float64(5), int64(1), object(7)
memory usage: 63.2+ KB


In [21]:
df_processed = df.copy()

# 1. Checking for missing values
print("\n1. Missing Values:")
missing_values = df_processed.isnull().sum()
if missing_values.sum()>0:
    print(missing_values[missing_values>0])
else:
    print("No missing values found (as expected from EDA)")

#2. Checking for duplicates
print("\n2. Duplicate Rows:")
duplicates = df_processed.duplicated().sum()
print(f"Number of duplicate rows: {duplicates}")
if duplicates > 0:
    print(f"Percentage of duplicates: {(duplicates/len(df_processed))*100:.2f}%")

#3. Checking for skewness for variable identified in EDA as right-skewed
print("\n3. Skewness Analysis (EDA identified right-skewed variables):")
skewed_vars = ["ApplicantIncome", "CoapplicantIncome", "LoanAmount", "LoanAmountTerm"]
for var in skewed_vars:
    if var in df_processed.columns:
        skewness = skew(df_processed[var])
        print(f"{var}: skewness = {skewness:.3f} ({'right-skewed' if skewness >0.5 else 'approximately normal'})")

#4. Checking for correlation with target (EDA evidence)
print("\n4. Correlation with Loan_Status (EDA Evidence):")
num_features = df_processed.select_dtypes(include=['float64', 'int64']).columns
# num_features = num_features.drop("Credit_History")
num_features = pd.DataFrame(df_processed[num_features])

correlations = num_features.corr()["Loan_Status"].sort_values(key=abs, ascending=False)
high_signal = correlations[abs(correlations)>0.2].drop("Loan_Status")
for feature, corr in high_signal.items():
    print(f"{feature}: {corr:.3f}")

print("\nLow-signal features (|correlation| < 0.1):")
low_signal = correlations[abs(correlations) < 0.1]
for feature, corr in low_signal.items():
    print(f"{feature}: {corr:.3f}")


1. Missing Values:
No missing values found (as expected from EDA)

2. Duplicate Rows:
Number of duplicate rows: 0

3. Skewness Analysis (EDA identified right-skewed variables):
ApplicantIncome: skewness = 6.494 (right-skewed)
CoapplicantIncome: skewness = 7.399 (right-skewed)
LoanAmount: skewness = 2.643 (right-skewed)

4. Correlation with Loan_Status (EDA Evidence):
Credit_History: 0.410

Low-signal features (|correlation| < 0.1):
CoapplicantIncome: -0.070
LoanAmount: -0.042
Loan_Amount_Term: -0.030
ApplicantIncome: -0.006


In [22]:
# num_features = df.select_dtypes(include=['float64', 'int64']).columns
# num_features = num_features.drop("Credit_History")
# num_features = pd.DataFrame(df[num_features])
# num_features

In [23]:
df_processed.columns

Index(['Loan_ID', 'Gender', 'Married', 'Dependents', 'Education',
       'Self_Employed', 'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area', 'Loan_Status'],
      dtype='object')

## 3. Handle Duplicates

In [24]:
if duplicates > 0:
    print(f"Removing {duplicates} duplicate rows...")
    df_processed = df_processed.drop_duplicates()
    print(f"Dataset shape after removing duplicates: {df_processed.shape}")
else:
    print("No duplicates to remove (as expected from EDA)")

No duplicates to remove (as expected from EDA)


## 5. Outlier Treatment (EDA Recommendation)


In [25]:
# Outlier treatment based on EDA recommendations
print("=== OUTLIER TREATMENT (IQR-CAPPING METHOD) ===")
print("EDA recommended IQR-capping for extreme features to preserve data points")

numerical_cols = df_processed.select_dtypes(include=[np.number]).columns.tolist()
if "Loan_Status_Int" in numerical_cols:
    numerical_cols.remove("Loan_Status_Int")
    numerical_cols.remove("Credit_History")
print(f"Treating outliers in {len(numerical_cols)} numerical features...")

#Applying IQR-capping method
outliers_capped = 0
for col in numerical_cols:
    Q1 = df_processed[col].quantile(0.25)
    Q3 = df_processed[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5*IQR
    upper_bound = Q3 + 1.5 * IQR

    #Counting outliers before capping
    outliers_before = ((df_processed[col]< lower_bound)| (df_processed[col] > upper_bound)).sum()
    if outliers_before > 0:
        #Cap outliers
        df_processed[col] = np.where(df_processed[col] < lower_bound, lower_bound, df_processed[col])
        df_processed[col] = np.where(df_processed[col]> upper_bound, upper_bound, df_processed[col])
        outliers_capped += outliers_before
        print(f"\n{col}: Capped {outliers_before} outliers")

print(f'\nTotal outliers capped: {outliers_capped}')
print(f"Dataset shape after outlier treatment: {df_processed.shape}")

=== OUTLIER TREATMENT (IQR-CAPPING METHOD) ===
EDA recommended IQR-capping for extreme features to preserve data points
Treating outliers in 6 numerical features...

ApplicantIncome: Capped 48 outliers

CoapplicantIncome: Capped 18 outliers

LoanAmount: Capped 39 outliers

Loan_Amount_Term: Capped 85 outliers

Credit_History: Capped 128 outliers

Total outliers capped: 318
Dataset shape after outlier treatment: (578, 13)


## 4. Log-Transform Skewed Variables (EDA Recommended)

In [26]:
print("=== LOG-TRANSFORMING SKEWED VARIABLES ===")
print("EDA identified these variables as right-skewed and recommended log transformation:")

#Variables to log-transform based on EDA findings
skewed_vars = ["ApplicantIncome", "CoapplicantIncome", "LoanAmount", "LoanAmountTerm"]

for var in skewed_vars:
    if var in df_processed.columns:
        #Check if variable has zero or negative values
        min_val = df_processed[var].min()
        if min_val <= 0:
            #Use log1p for variables with zeros
            df_processed[f"{var}_log"] = np.log1p(df_processed[var])
            print(f"\n{var}: Applied log1p transformation (had {min_val:.3f} minimum value)")
        else:
            #use log for positive values only
            df_processed[f"{var}_log"]= np.log(df_processed[var])
            print(f"\n{var}: Applied log transformation")
        
        #Checking skewness before and after
        original_skew = skew(df_processed[var])
        transformed_skew = skew(df_processed[f"{var}_log"])
        print(f"Original skewness: {original_skew:.3f} → Transformed skewness: {transformed_skew:.3f}")

print(f"\n Dataset shape after log transformation: {df_processed.shape}")
print(f"New log-transformed columns:", [col for col in df_processed.columns if '_log' in col])

=== LOG-TRANSFORMING SKEWED VARIABLES ===
EDA identified these variables as right-skewed and recommended log transformation:

ApplicantIncome: Applied log transformation
Original skewness: 1.034 → Transformed skewness: -0.636

CoapplicantIncome: Applied log1p transformation (had 0.000 minimum value)
Original skewness: 1.015 → Transformed skewness: -0.190

LoanAmount: Applied log transformation
Original skewness: 0.708 → Transformed skewness: -0.885

 Dataset shape after log transformation: (578, 16)
New log-transformed columns: ['ApplicantIncome_log', 'CoapplicantIncome_log', 'LoanAmount_log']


Bsed on EDA findings, handle outliers using IQR-capping method

## 6. Feature Engineering

In [27]:
df_processed.columns

Index(['Loan_ID', 'Gender', 'Married', 'Dependents', 'Education',
       'Self_Employed', 'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area', 'Loan_Status',
       'ApplicantIncome_log', 'CoapplicantIncome_log', 'LoanAmount_log'],
      dtype='object')

In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 578 entries, 1 to 613
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            578 non-null    object 
 1   Gender             578 non-null    object 
 2   Married            578 non-null    object 
 3   Dependents         578 non-null    object 
 4   Education          578 non-null    object 
 5   Self_Employed      578 non-null    object 
 6   ApplicantIncome    578 non-null    float64
 7   CoapplicantIncome  578 non-null    float64
 8   LoanAmount         578 non-null    float64
 9   Loan_Amount_Term   578 non-null    float64
 10  Credit_History     578 non-null    float64
 11  Property_Area      578 non-null    object 
 12  Loan_Status        578 non-null    int64  
dtypes: float64(5), int64(1), object(7)
memory usage: 63.2+ KB


In [29]:

num_col = df_processed.select_dtypes(include=['float64', 'int64']).columns


### Encoding

In [30]:
df_processed.head(2)

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,ApplicantIncome_log,CoapplicantIncome_log,LoanAmount_log
1,LP001003,Male,Yes,1,Graduate,No,4583.0,1508.0,128.0,360.0,1.0,Rural,0,8.430109,7.319202,4.852030
2,LP001005,Male,Yes,0,Graduate,Yes,3000.0,0.0,66.0,360.0,1.0,Urban,1,8.006368,0.000000,4.189655


In [31]:
#Encoding 
le = LabelEncoder()
binary_col = ["Gender", "Married", "Education", "Self_Employed"]
for col in binary_col:
    df[col] = le.fit_transform(df_processed[col])

# df[["Gender", "Married", "Education", "Self_Employed"]]
# df.head(2)

ohe = OneHotEncoder(sparse_output=False, drop="first")
df_processed["Property_Area"] = ohe.fit_transform(df[["Property_Area"]])
df_processed.head(2)

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,ApplicantIncome_log,CoapplicantIncome_log,LoanAmount_log
1,LP001003,Male,Yes,1,Graduate,No,4583.0,1508.0,128.0,360.0,1.0,0.0,0,8.430109,7.319202,4.852030
2,LP001005,Male,Yes,0,Graduate,Yes,3000.0,0.0,66.0,360.0,1.0,0.0,1,8.006368,0.000000,4.189655


In [32]:
df_processed.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,ApplicantIncome_log,CoapplicantIncome_log,LoanAmount_log
1,LP001003,Male,Yes,1,Graduate,No,4583.0,1508.0,128.0,360.0,1.0,0.0,0,8.430109,7.319202,4.852030
2,LP001005,Male,Yes,0,Graduate,Yes,3000.0,0.0,66.0,360.0,1.0,0.0,1,8.006368,0.000000,4.189655
3,LP001006,Male,Yes,0,Not Graduate,No,2583.0,2358.0,120.0,360.0,1.0,0.0,1,7.856707,7.765993,4.787492
4,LP001008,Male,No,0,Graduate,No,6000.0,0.0,141.0,360.0,1.0,0.0,1,8.699515,0.000000,4.948760
5,LP001011,Male,Yes,2,Graduate,Yes,5417.0,4196.0,267.0,360.0,1.0,0.0,1,8.597297,8.342125,5.587249


### Feature Scaling

In [33]:
#Scaling
scaler = StandardScaler()
df_processed[num_col] = scaler.fit_transform(df_processed[num_col])
df_processed[num_col].std().mean()

np.float64(0.7784514702191799)

In [34]:
print("FEATURE ENGINEERING BASED ON EDA RECOMMENDATIONS")

#Splitting income into bins
df_processed["Income_Category"] = pd.qcut(df_processed["ApplicantIncome"], 3, labels=False)

#Total loan applicant income
df_processed["Total_Income"] = df_processed["ApplicantIncome"] + df_processed["CoapplicantIncome"]

#Loan to income ratio
df_processed["Loan_Income_Ratio"] = df_processed["LoanAmount"]/df_processed["Total_Income"]

#Estimated monthly payment plan
df_processed["Monthly_Payment"] = df_processed["LoanAmount"]/df_processed["Loan_Amount_Term"]

df_processed[["Total_Income", "LoanAmount","Loan_Income_Ratio", "Monthly_Payment", "Income_Category"]].head()

FEATURE ENGINEERING BASED ON EDA RECOMMENDATIONS


,Total_Income,LoanAmount,Loan_Income_Ratio,Monthly_Payment,Income_Category
1,0.034966,-0.185363,-5.301191,-inf,1
2,-1.528333,-1.247172,0.816034,-inf,0
3,-0.249704,-0.322371,1.291013,-inf,0
4,-0.319070,0.037274,-0.116821,inf,2
5,2.020325,2.195143,1.086530,inf,2


## 7. Feature Selection


In [35]:
df_processed

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,ApplicantIncome_log,CoapplicantIncome_log,LoanAmount_log,Income_Category,Total_Income,Loan_Income_Ratio,Monthly_Payment
1,LP001003,Male,Yes,1,Graduate,No,-0.015883,0.050849,-0.185363,0.0,0.0,0.0,-1.517517,0.239578,0.788442,0.030445,1,0.034966,-5.301191,-inf
2,LP001005,Male,Yes,0,Graduate,Yes,-0.653971,-0.874362,-1.247172,0.0,0.0,0.0,0.658971,-0.534381,-1.113273,-1.408405,0,-1.528333,0.816034,-inf
3,LP001006,Male,Yes,0,Not Graduate,No,-0.822058,0.572354,-0.322371,0.0,0.0,0.0,0.658971,-0.807734,0.904530,-0.109749,0,-0.249704,1.291013,-inf
4,LP001008,Male,No,0,Graduate,No,0.555292,-0.874362,0.037274,0.0,0.0,0.0,0.658971,0.731643,-1.113273,0.240566,2,-0.319070,-0.116821,inf
5,LP001011,Male,Yes,2,Graduate,Yes,0.320292,1.700033,2.195143,0.0,0.0,0.0,0.658971,0.544945,1.054224,1.627528,2,2.020325,1.086530,inf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,LP002978,Female,No,0,Graduate,No,-0.694279,-0.874362,-1.161542,0.0,0.0,0.0,0.658971,-0.596301,-1.113273,-1.249776,0,-1.568642,0.740476,-inf
610,LP002979,Male,Yes,3+,Graduate,No,-0.208156,-0.874362,-1.692446,0.0,0.0,0.0,0.658971,0.038838,-1.113273,-2.496218,1,-1.082518,1.563435,-inf
611,LP002983,Male,Yes,1,Graduate,No,1.390490,-0.727114,1.955380,0.0,0.0,0.0,0.658971,1.273456,0.311817,1.510533,2,0.663376,2.947619,inf
612,LP002984,Male,Yes,2,Graduate,No,1.193380,-0.874362,0.825068,0.0,0.0,0.0,0.658971,1.159314,-1.113273,0.853901,2,0.319018,2.586275,inf


In [36]:
# Feature selection based on EDA correlation evidence
print("=== FEATURE SELECTION BASED ON EDA CORRELATION EVIDENCE ===")

X = df_processed.drop(columns=["Loan_Status", "Loan_ID"], axis=1)
y = df_processed["Loan_Status"]

print(f"Total features available: {X.shape[1]}")

#EDA identified high-signal features (|correlation|>0.2)
high_signal_features = ["Credit_History"]
print(f"\nEDA-identified high-signal features: {high_signal_features}")

# EDA-identified low-signal features (|correlation| < 0.1) 
low_signal_features = ["CoapplicantIncome", "LoanAmount", "Loan_Amount_Term", "ApplicantIncome"]
print(f"EDA-identified low-signal features: {low_signal_features}")

# Check which features are actually available
available_high_signal = [f for f in high_signal_features if f in X.columns]
available_low_signal = [f for f in low_signal_features if f in X.columns]

print(f"\nAvailable high-signal features: {available_high_signal}")
print(f"Available low-signal features: {available_low_signal}")

# Create feature sets for evaluation
print("\n=== FEATURE SETS FOR EVALUATION ===")

#Set1: All original features (excluding low signal)
features_exclude_low_signal = [col for col in X.columns if col not  in low_signal_features]
print(f"Set 1 - Exclude low-signal features; {len(features_exclude_low_signal)} features")

# Set 2: High-signal + engineered features
high_signal_plus_engineered = available_high_signal + [col for col in X.columns if any(keyword in col.lower() for keyword in ["total", "ratio", "monthly", "log"])]
print(f"Set 2 - High-signal + engineered: {len(high_signal_plus_engineered)} features")

#Set 3: All features (for comparison)
all_features = list(X.columns)
print(f"Set 3 - All features: {len(all_features)} features")

#Using high signal features and engineered features as recommended by the EDA
selected_features = high_signal_plus_engineered
print(f"\n✓ Selected feature set: {len(selected_features)} features")
print("Selected features:", selected_features)


=== FEATURE SELECTION BASED ON EDA CORRELATION EVIDENCE ===
Total features available: 18

EDA-identified high-signal features: ['Credit_History']
EDA-identified low-signal features: ['CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term', 'ApplicantIncome']

Available high-signal features: ['Credit_History']
Available low-signal features: ['CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term', 'ApplicantIncome']

=== FEATURE SETS FOR EVALUATION ===
Set 1 - Exclude low-signal features; 14 features
Set 2 - High-signal + engineered: 7 features
Set 3 - All features: 18 features

✓ Selected feature set: 7 features
Selected features: ['Credit_History', 'ApplicantIncome_log', 'CoapplicantIncome_log', 'LoanAmount_log', 'Total_Income', 'Loan_Income_Ratio', 'Monthly_Payment']


## 8. Data Splitting (Stratified)

In [37]:
#Stratified data splitting based on EDA findsings about class imbalance
print("=== STRATIFIED DATA SPLITTING ===")
print("EDA identified class imbalance - using stratified splitting to preserve class distribution")

#Select chosen features
X_selected = X[selected_features]
print(f"Selected features shape: {X_selected.shape}")

#First split: 80% train+val, 20% test
X_temp, X_test, y_temp, y_test = train_test_split(
    X_selected, y,  test_size=0.2, random_state=42, stratify=y
)

#Second split: 75% train, 25% validation (of the 80%)
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.25, random_state=42, stratify = y_temp
)
print(f"\nData split results:")
print(f"Training set: {X_train.shape} ({(X_train.shape[0]/len(X_selected))*100:.1f})%")
print(f"Validation set: {X_val.shape} ({(X_val.shape[0]/len(X_selected))*100:.1f})%")
print(f"Test set: {X_test.shape} ({(X_test.shape[0]/len(X_selected))*100:.1f})%")

# Check class distribution in each set (should be similar due to stratification)
print(f"\nClass distribution verification:")
print("Training set quality distribution:")
print(y_train.value_counts().sort_index())

print("\nValidation set quality distribution:")
print(y_val.value_counts().sort_index())

print("\nTest set quality distribution:")
print(y_test.value_counts().sort_index())


=== STRATIFIED DATA SPLITTING ===
EDA identified class imbalance - using stratified splitting to preserve class distribution
Selected features shape: (578, 7)

Data split results:
Training set: (346, 7) (59.9)%
Validation set: (116, 7) (20.1)%
Test set: (116, 7) (20.1)%

Class distribution verification:
Training set quality distribution:
Loan_Status
-1.517517    105
 0.658971    241
Name: count, dtype: int64

Validation set quality distribution:
Loan_Status
-1.517517    35
 0.658971    81
Name: count, dtype: int64

Test set quality distribution:
Loan_Status
-1.517517    35
 0.658971    81
Name: count, dtype: int64


## 9. Feature Scaling

In [38]:
#Applying StandardScaler as recommended by EDA
print("=== FEATURE SCALING (STANDARD SCALER) ===")
print("EDA recommended StandardScaler for distance-based models")

#Fitting scaler on training data only (to avoid data leakage)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)

#Transforming validation and test sets
X_val_scaled = scaler.transform(X_val)
X_val_scaled = pd.DataFrame(X_val_scaled, columns=X_val.columns)

X_test_scaled = scaler.transform(X_test)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)

print("✓ Scaling applied successfully!")
print(f"Training set scaled - Mean: {X_train_scaled.mean().mean():.4f}, Std: {X_train_scaled.std().mean():.4f}")
print(f"Validation set scaled - Mean: {X_val_scaled.mean().mean():.4f}, Std: {X_val_scaled.std().mean():.4f}")
print(f"Test set scaled - Mean: {X_test_scaled.mean().mean():.4f}, Std: {X_test_scaled.std().mean():.4f}")

#Verigy scaling worked correctly
print(f"\nScaling verification")
print(f"Training set - Mean ≈ 0: {abs(X_train_scaled.mean().mean())< 0.01}")
print(f"Training set - Std ≈ 1: {abs(X_train_scaled.std().mean()-1) < 0.01}")
print(f"{abs(X_train_scaled.std().mean())}")

=== FEATURE SCALING (STANDARD SCALER) ===
EDA recommended StandardScaler for distance-based models


ValueError: Input X contains infinity or a value too large for dtype('float64').